In [1]:
%pip install highway-env

# Environment
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

import highway_env



# Models and computation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import namedtuple

# Visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import trange

# IO
from pathlib import Path

/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


In [2]:
import sys
from tqdm.notebook import trange
from utils import record_videos, show_videos

In [3]:

n_cpu = 6
batch_size = 64
env = make_vec_env("parking-v0", n_envs=n_cpu, vec_env_cls=SubprocVecEnv)
model = PPO(
    "MultiInputPolicy",
    env,
    policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    n_steps=batch_size * 12 // n_cpu,
    batch_size=batch_size,
    n_epochs=10,
    learning_rate=5e-4,
    gamma=0.8,
    verbose=2,
    tensorboard_log="parking_ppo/",
)
# Train the agent
model.learn(total_timesteps=int(1e4))
# Save the agent
model.save("parking_ppo/model")

model = PPO.load("parking_ppo/model")
# env = gym.make("parking-v0", render_mode = "rgb_array")
# for _ in range(5):
#     obs, info = env.reset()
#     done = truncated = False
#     while not (done or truncated):
#         action, _ = model.predict(obs)
#         obs, reward, done, truncated, info = env.step(action)
#         env.render()

/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/pytho

Using cpu device


/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Logging to parking_ppo/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 42.5     |
|    ep_rew_mean     | -27.6    |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 1346     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 768      |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 52.5        |
|    ep_rew_mean          | -30.5       |
|    success_rate         | 0           |
| time/                   |             |
|    fps                  | 822         |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 1536        |
| train/                  |             |
|    approx_kl            | 0.011714739 |
|    clip_fraction        | 0.129       |
|    clip_range           | 0.2         |
|    entr

In [5]:
model = PPO.load("parking_ppo/model")
env = gym.make("parking-v0", render_mode="rgb_array")
env = record_videos(env)
env.reset()
done = False
while not done:
    action = model.predict(obs)[0]
    obs, reward, done, truncated, info = env.step(action)
env.close()
show_videos()

/Users/noahwiley-class/.pyenv/versions/3.10.1/envs/6.8200/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/noahwiley-class/Documents/6.820/Final/language-reward-design/wiley/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /Users/noahwiley-class/Documents/6.820/Final/language-reward-design/wiley/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/noahwiley-class/Documents/6.820/Final/language-reward-design/wiley/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/noahwiley-class/Documents/6.820/Final/language-reward-design/wiley/videos/rl-video-episode-0.mp4


In [ ]:
print(env)
# fig, ax = plt.subplots(ncols=1, figsize=(12, 5))
# ax.imshow(render) # , cmap=plt.get_cmap('gray')
# plt.show()